In [10]:
%pip install spacy nltk
!python -m spacy download es_core_news_md
import nltk
import spacy
from glob import glob
spacy_es = spacy.load("es_core_news_md") #Spanish corpus available in Spacy
spacy_es.max_length = 4000000
nltk.download("punkt")
nltk.download("cess_esp") #Spanish corpus available in NLTK

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 42.3/42.3 MB 13.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pilyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\pilyu\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

In [11]:
#Read the file
def read_file(filename):
    with open(filename, encoding='utf8') as infile:
        contents = infile.read()
    return contents

In [26]:
#Some words are split into individual letters, and this function tries to recompose those words, removing the spaces.
def retokenize(tokenized_text):
    tokenized = []
    is_broken_word = False
    temp_word = ""
    for token in tokenized_text:
        token_lower = token.lower()
        if len(token) == 1:
            if not is_broken_word:
                is_broken_word = True
                temp_word = token_lower
            else:
                temp_word += token_lower
        else:
            if is_broken_word:
                tokenized.append(temp_word)
                temp_word = ""
                is_broken_word = False
            tokenized.append(token_lower)
    return tokenized

In [27]:
#Tokenize all the texts
tokens = set()
for filename in glob('texts/*.txt'):
    text = read_file(filename)
    tokenized_text = nltk.tokenize.word_tokenize(text)
    tokenized_text = set(retokenize(tokenized_text))
    tokens = tokens | tokenized_text

In [28]:
#Words recognized by each corpus
corpora = {}
words_corpus = {word.lower() for word in nltk.corpus.cess_esp.words()}
corpora["cess_esp"] = {word for word in tokens if word in words_corpus}
doc = spacy_es(" ".join(tokens))
corpora["es_core_news_md"] = {token.text for token in doc if token.has_vector}

In [29]:
#Calculate the percentage of words recognized 
for corpus, tokens_recognized in corpora.items():
    percent_score = len(tokens_recognized)*100/ len(tokens)
    print(f"{corpus} {percent_score}%")

cess_esp 4.1495567091325665%
es_core_news_md 18.228607480049146%
